<a href="https://colab.research.google.com/github/srikanthrc/running-to-stand-still/blob/master/_notebooks/2020-03-26-covid19_compare_us_trajectories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare State Trajectories - Total Cases
> Comparing state trajectories of total cases 

- comments: false
- author: Srikanth Chinmay (inspired by Pratap Vardhan @ covid19dashboards.com) 
- categories: [covid-19]
- image: images/covid-compare-state-trajectories.png
- permalink: /compare-state-trajectories/

```
# This is formatted as code
```



In [0]:
#hide
import pandas as pd
import altair as alt
from IPython.display import HTML

In [0]:
#hide
from pathlib import Path
if not Path('covid_overview.py').exists():
    ! wget https://raw.githubusercontent.com/srikanthrc/covid-19/master/covid_overview.py

In [0]:
#hide
import covid_overview as covid

In [75]:
#hide

# get Countries data from JHU
url = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
       'csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df = pd.read_csv(url)
# rename countries
df['Country/Region'] = df['Country/Region'].replace({'Korea, South': 'South Korea'})
df = df[~df['Country/Region'].isin(['Cruise Ship'])]   # Remove Ships

dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]

dff = (df.groupby('Country/Region')[dt_cols].sum()
       .stack().reset_index(name='Confirmed Cases')
       .rename(columns={'level_1': 'Date', 'Country/Region': 'Key', 'Confirmed Cases': 'Cases'}))
dff['Date'] = pd.to_datetime(dff['Date'], format='%m/%d/%y')
display(dff.head(5))

,Key,Date,Cases
0,Afghanistan,2020-01-22,0
1,Afghanistan,2020-01-23,0
2,Afghanistan,2020-01-24,0
3,Afghanistan,2020-01-25,0
4,Afghanistan,2020-01-26,0


In [76]:
#hide
# get States data from covidtracking
df_s = covid.get_covidtracking_data('positive')
dt_cols_s = df_s.columns[~df_s.columns.isin(['Province/State', 'Country/Region','Lat','Long'])]

dff_s = (df_s.groupby('Province/State')[dt_cols_s].sum()
          .stack().reset_index(name='positive')
          .rename(columns={'level_1': 'Date', 'Province/State': 'Key', 'positive': 'Cases'}))
dff_s['Date'] = pd.to_datetime(dff_s['Date'], format="%m/%d/%y")
display(dff_s.head(5))

,Key,Date,Cases
0,Alabama,2020-03-04,0.0
1,Alabama,2020-03-05,0.0
2,Alabama,2020-03-06,0.0
3,Alabama,2020-03-07,0.0
4,Alabama,2020-03-08,0.0


In [0]:
#hide
MIN_CASES = 300
LAST_DATE = dt_cols[-1]
# sometimes last column may be empty, then go backwards
for c in dt_cols[::-1]:
    if not df[c].fillna(0).eq(0).all():
        LAST_DATE = c
        break

countries = dff[dff['Date'].eq(LAST_DATE) & dff['Cases'].ge(MIN_CASES) & 
        dff['Key'].isin(('Japan','South Korea','Italy','Germany','Spain','United Kingdom','US','India'))
       ].sort_values(by='Cases', ascending=False)
countries = countries['Key'].values

states = dff_s[dff_s['Date'].eq(LAST_DATE) & dff_s['Cases'].ge(MIN_CASES)].sort_values(by='Cases', ascending=False)
states = states['Key'].values

In [0]:
#hide
SINCE_CASES_NUM = 100
dff2 = dff[dff['Key'].isin(countries)].copy()
days_since = (dff2.assign(F=dff2['Cases'].ge(SINCE_CASES_NUM))
              .set_index('Date')
              .groupby('Key')['F'].transform('idxmax'))
dff2['Days since 100 cases'] = (dff2['Date'] - days_since.values).dt.days.values
dff2 = dff2[dff2['Days since 100 cases'].ge(0)]

In [0]:
#hide
days_since = (dff_s.assign(F=dff_s['Cases'].ge(SINCE_CASES_NUM))
              .set_index('Date')
              .groupby('Key')['F'].transform('idxmax'))
dff_s['Days since 100 cases'] = (dff_s['Date'] - days_since.values).dt.days.values

dff_s2 = dff_s[dff_s['Days since 100 cases'].ge(0)]

In [0]:
#hide
def get_colors(x):
    mapping = {
        'Italy': 'grey',
        'Iran': '#A1BA59',
        'Japan': '#9467bd',
        'South Korea': '#E45756',
        'Spain': '#F58518',
        'Germany': '#2495D3',
        'United Kingdom': '#2495D3',
        'US': '#3469B2',
        'India': '#9D755D'}
    return mapping.get(x, '#3469B2')

In [0]:
#hide_input
baseline_countries = ['Italy', 'South Korea', 'Japan']
max_date = dff2['Date'].max()
color_domain = list(dff2['Key'].unique())
color_range = list(map(get_colors, color_domain))

def make_since_chart(highlight_countries=[], baseline_countries=baseline_countries):
    selection = alt.selection_multi(fields=['Key'], bind='legend', 
                                    init=[{'Key': x} for x in highlight_countries + baseline_countries])

    base = alt.Chart(dff2, width=550).encode(
        x='Days since 100 cases:Q',
        y=alt.Y('Cases:Q', scale=alt.Scale(type='log')),
        color=alt.Color(
            'Key:N',
            scale=alt.Scale(domain=color_domain, range=color_range),
            legend=alt.Legend(columns=2)),
        tooltip=list(dff2),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
    )
    max_day = dff2['Days since 100 cases'].max()
    ref = pd.DataFrame([[x, 100*1.33**x] for x in range(max_day+1)], columns=['Days since 100 cases', 'Cases'])
    base_ref = alt.Chart(ref).encode(x='Days since 100 cases:Q', y='Cases:Q')
    return (
        base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3]) +
        base_ref.transform_filter(
            alt.datum['Days since 100 cases'] >= max_day
        ).mark_text(dy=-6, align='right', fontSize=10, text='33% Daily Growth') +
        base.mark_line(point=True).add_selection(selection) + 
        base.transform_filter(
            alt.datum['Date'] >= int(max_date.timestamp() * 1000)
        ).mark_text(dy=-8, align='right', fontWeight='bold').encode(text='Key:N')
    ).properties(
        title=f"Compare {', '.join(highlight_countries)} trajectory with {', '.join(baseline_countries)}"
    )

## Learning from Italy, South Korea & Japan

Italy, South Korea & Japan are three countries which show different growth rates and how it evolved over time. 

**South Korea** flattened it's growth after 2 weeks since 100 cases. **Italy** continue to grew after 3rd week.

Where does a Country of interest stand today?

<small>Click (Shift+ for multiple) on Countries legend to filter the visualization.</small>

In [82]:
#hide_input
HTML(f'<small class="float-right">Last Updated on {pd.to_datetime(LAST_DATE).strftime("%B, %d %Y")}</small>')

In [83]:
#hide_input
chart = make_since_chart(highlight_countries=['US', 'India'])
chart

alt.LayerChart(...)

{{LAST_DATE}}

In [84]:
#hide 
dff3 = pd.concat([dff2, dff_s2])
dff3

,Key,Date,Cases,Days since 100 cases
4007,Germany,2020-03-01,130.0,0
4008,Germany,2020-03-02,159.0,1
4009,Germany,2020-03-03,196.0,2
4010,Germany,2020-03-04,262.0,3
4011,Germany,2020-03-05,482.0,4
...,...,...,...,...
1283,Wyoming,2020-03-22,24.0,18
1284,Wyoming,2020-03-23,26.0,19
1285,Wyoming,2020-03-24,29.0,20
1286,Wyoming,2020-03-25,44.0,21


Select a state from the drop down list below to toggle  the visualization.

In [85]:
#hide
color_domain = list(dff3['Key'].unique())
color_range = list(map(get_colors, color_domain))
max_date = dff3['Date'].max()

base = alt.Chart(dff3, width=600).encode(
    x='Days since 100 cases:Q',
    y=alt.Y('Cases:Q', scale=alt.Scale(type='log')),
    color=alt.Color('Key:N', scale=alt.Scale(domain=color_domain, range=color_range), legend=None),
    tooltip=['Key', 'Date', 'Cases', 'Days since 100 cases']
)

country_selection = alt.selection_single(
    name='Select', fields=['Key'], 
    bind=alt.binding_select(options=list(sorted(set(countries) - set(baseline_countries)))),
    init={'Key': 'US'})

state_selection = alt.selection_single(
    name='Select', fields=['Key'], 
    bind=alt.binding_select(options=list(sorted(set(states) - set(baseline_countries)))),
    init={'Key': 'New York'})

date_filter = alt.datum['Date'] >= int(max_date.timestamp() * 1000)

max_day = dff3['Days since 100 cases'].max()
ref = pd.DataFrame([[x, 100*1.33**x] for x in range(max_day+1)], columns=['Days since 100 cases', 'Cases'])
base_ref = alt.Chart(ref).encode(x='Days since 100 cases:Q', y='Cases:Q')
base_ref_f = base_ref.transform_filter(alt.datum['Days since 100 cases'] >= max_day)

display(date_filter)

(datum['Date'] >= 1585180800000)

In [0]:
#hide
base2 = base.transform_filter(alt.FieldOneOfPredicate('Key', baseline_countries))
base3 = base.transform_filter(state_selection)
base4 = base3.transform_filter(date_filter)

In [89]:
#hide_input
chart5 = (
 base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3])
 + base_ref_f.mark_text(dy=-6, align='right', fontSize=10, text='33% Daily Growth') 
 + base2.mark_line(point=True, tooltip=True)
 + base2.transform_filter(date_filter).mark_text(dy=-8, align='right').encode(text='Key:N')
 + base3.mark_line(point={'size':30}, tooltip=True)
 + base4.mark_text(dx=8, align='left', fontWeight='bold').encode(text='Key:N')
 + base4.mark_text(dx=8, dy=12, align='left', fontWeight='bold').encode(text='Cases:Q')
).add_selection(state_selection).properties(
    title=f"State's Trajectory compared to {', '.join(baseline_countries)}"
)
chart5

alt.LayerChart(...)

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Sources: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub](https://github.com/CSSEGISandData/COVID-19). [covidtracking.com](https://covidtracking.com/)